A solution:

KNO3 0.15841 moles in 1L of A mix =>
conc of KNO3 = 0.158 mol/L =>
moles of KNO3 in 10mL = 0.00158 =>
Assuming main tank capacity as 10L,
increase in the concentration of K in the main tank is 0.000158mol/L = 6.18 ppm.

For 1ml, increase in K is 0.618 ppm.

For N, 1ml results in increase of 1.1424 ppm.

B Solution:

For N, 1ml => 0.591 ppm

For P, 0.38812 ppm

For K, 1.517 ppm

In [ ]:
from math import gcd

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def find_npkratio(n, p, k):
    n = int(n)
    p = int(p)
    k = int(k)
    # Find the greatest common divisor (GCD) of n, p, and k
    gcd_npk = gcd(gcd(n, p), k)

    # Divide each element by the GCD to get the ratio
    n_ratio = n // gcd_npk
    p_ratio = p // gcd_npk
    k_ratio = k // gcd_npk

    return (n_ratio, p_ratio, k_ratio)

In [ ]:
pip install stable-baselines3

  Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.0.0a2
    Uninstalling gymnasium-1.0.0a2:
      Successfully uninstalled gymnasium-1.0.0a2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shimmy 2.0.0 requires gymnasium>=1.0.0a1, but you have gymnasium 0.29.1 which is incompatible.


In [ ]:
!pip install 'shimmy>=0.2.1'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.3/954.3 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.1
    Uninstalling gymnasium-0.29.1:
      Successfully uninstalled gymnasium-0.29.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 2.3.2 requires gymnasium<0.30,>=0.28.1, but you have gymnasium 1.0.0a2 which is incompatible.


In [ ]:
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env


class GreenhouseEnv(gym.Env):
    def __init__(self):
        super(GreenhouseEnv, self).__init__()
        # State space: N, P, K levels and pH
        self.observation_space = gym.spaces.Box(low=np.array([0, 0, 0, 3]), high=np.array([100, 100, 100, 11]), dtype=np.float32)
        # Action space: 5 actions and 11 discrete amounts (0, 10, 20, ..., 100 ml)
        self.action_space = gym.spaces.MultiDiscrete([4, 11])
        self.state = None  # Initial N, P, K, pH levels will be set in reset()

    def step(self, action):
        N_A = 1.1424
        K_A = 0.618
        N_B = 0.591
        P_B = 0.38812
        K_B = 1.517
        conc = 10**(-3) # Concentration of acid used (To be Updated)
        vol_of_tank = 10 # To be Updated

        # Extract discrete action and amount (amount is in steps of 10 ml)
        discrete_action = int(action[0])
        amount_step = int(action[1])
        amount = amount_step * 10

        # Initialize inputs
        a_solution = b_solution = acid_input = 0

        # Perform the action
        if discrete_action == 1:
            self.state[0] += amount * N_A
            self.state[2] += amount * K_A
            a_solution = amount

        elif discrete_action == 2:
            self.state[0] += amount * N_B # To be Updated
            self.state[1] += amount * P_B
            self.state[2] += amount * K_B
            b_solution = amount

        elif discrete_action == 3:
            curr_conc = 10**(-self.state[3])
            new_conc = curr_conc +  (conc * amount / vol_of_tank)
            pH = -1*np.log10(new_conc)
            self.state[3] = pH  # Increase pH by adding acid
            acid_input = amount

        # elif discrete_action == 4:
        #     self.state[3] += amount * 0.01  # Decrease pH by adding base

        # Clip the state to stay within valid ranges
        self.state = np.clip(self.state, [0, 0, 0, 0], [100, 100, 100, 14])

        # Calculate reward
        reward = -(a_solution + b_solution + acid_input)

        # Check if N, P, K levels are within optimal range
        if 40 < self.state[0] < 60 and 30 < self.state[1] < 50 and 35 < self.state[2] < 55 and 5.5 < self.state[3] < 7.0:
            reward += 50
        if 40 < self.state[0] < 60 :
            reward += 5
        if 30 < self.state[1] < 50 :
            reward += 5
        if 35 < self.state[2] < 55 :
            reward += 5
        if 5.5 < self.state[3] < 7.0 :
            reward += 10

        # Penalty for extreme NPK levels
        if self.state[0] < 20 or self.state[0] > 80 or self.state[1] < 20 or self.state[1] > 70 or self.state[2] < 25 or self.state[2] > 75 or self.state[3] < 5.5 or self.state[3] > 7.0:
            reward -= 30

        # I found that the optimal NPK ratio is 3:1:2
        a,b,c = find_npkratio(self.state[0], self.state[1], self.state[2])
        NPK_ratio_penalty = (a - 3)**2 + (b - 1)**2 + (c - 2)**2
        reward -= NPK_ratio_penalty

        if a==3 and b==1 and c==2: # Ratio can be maintained even if doesn't have enough nutrients. So, I have considered less reward.
            reward += 10

        done = 40 < self.state[0] < 60 and 30 < self.state[1] < 50 and 35 < self.state[2] < 55 and 5.5 < self.state[3] < 7.0
        return self.state, reward, done, {}

    def reset(self):
        # Randomize initial state within a realistic range
        self.state = np.array([
            np.random.uniform(15, 85),  # N level
            np.random.uniform(15, 75),  # P level
            np.random.uniform(20, 80),  # K level
            np.random.uniform(4.0, 10.0) # pH level
        ])
        return self.state

    def render(self, mode='human'):
        pass

env = make_vec_env(lambda: GreenhouseEnv(), n_envs=1)

# Define the PPO model
model = PPO('MlpPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=30000)

# Test the trained agent
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()


/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


Using cuda device
-----------------------------
| time/              |      |
|    fps             | 448  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 335         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009557802 |
|    clip_fraction        | 0.097       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.78       |
|    explained_variance   | 9.89e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.25e+05    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0103     |
|    value_loss           | 9.15e+07    |
-----------------------------------------
----------------

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")
